# Mushrooms

In [25]:
import pandas as pd
import numpy as np
import random
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

mushrooms = pd.read_csv("agaricus-lepiota_h.data")
for v in mushrooms.columns:
    mushrooms[v] = mushrooms[v].fillna('z')
mushrooms.head()

In [26]:
mushroomsTarget = mushrooms["poisonous"]
mushroomsTarget.describe()

In [27]:
mushroomsFeatures = mushrooms.drop(columns = "poisonous")
mushroomsFeatures.describe()

In [28]:
binary_columns = [col for col in mushroomsFeatures.columns if mushroomsFeatures[col].nunique() == 2]
mushroomsFeatures = pd.get_dummies(mushroomsFeatures, columns=[col for col in mushroomsFeatures.columns if col not in binary_columns], drop_first=False)
mushroomsFeatures = pd.get_dummies(mushroomsFeatures, columns=binary_columns, drop_first=True)
mushroomsTarget = pd.get_dummies(mushroomsTarget, columns=['poisonous'], drop_first=True)

In [29]:
mushroomsTarget.describe()

In [30]:
mushroomsFeatures.describe()

In [31]:

X_train, X_test, y_train, y_test = train_test_split(mushroomsFeatures, mushroomsTarget, test_size=0.2, random_state=0)

In [32]:

classifier_input = keras.Input(shape=(X_train.shape[1],))
x = keras.layers.Dense(
    units=16,
    activation='relu',
    use_bias=True,
    bias_initializer="zeros",
)(classifier_input)
x = keras.layers.Dense(
    units=8,
    activation='relu',
    use_bias=True,
    bias_initializer="zeros",
)(x)
classifier_output = keras.layers.Dense(
    units=1,
    activation='relu',
    use_bias=True,
    bias_initializer="zeros",
)(x)
classifier = keras.Model(inputs=classifier_input, outputs=classifier_output)


In [33]:
classifier.summary()

In [34]:
classifier.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    # loss=keras.losses.BinaryCrossentropy(),
    loss=keras.losses.MSE,
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Recall(),
        keras.metrics.Precision(),
        keras.metrics.AUC()
    ],
)

In [35]:
classifier.evaluate(
    x=X_test.astype('int').values,
    y=y_test.astype('int').values,
    batch_size=None,
    verbose=1,
)

In [36]:
history = classifier.fit(
    x=X_train.astype('int').values,
    y=y_train.astype('int').values,
    batch_size=None,
    epochs=10,
    shuffle=True,
    steps_per_epoch=None,
)

In [37]:
for k in history.history.keys():
  plt.plot(history.history[k])
plt.legend(history.history.keys(), loc='upper left')

# Laptop

In [38]:
laptop = pd.read_csv("Laptop_price.csv")
laptop.describe()

In [39]:
priceMin = laptop["Price"].min()
priceMax = laptop["Price"].max()
def unnorm_price(price):
  return price * (priceMax - priceMin) + priceMin
fractional = ['Processor_Speed', 'Screen_Size', 'Weight', 'Price']
for col in [c for c in laptop.columns if c not in ["Brand"]]:
    laptop[col] = ((laptop[col] - laptop[col].min()) / (laptop[col].max() - laptop[col].min()))
laptop = pd.get_dummies(laptop, columns=["Brand"], drop_first=False)
laptop.head()

In [40]:

laptopFeatures = laptop.drop("Price", axis=1)
laptopTarget = laptop["Price"]


X_train, X_test, y_train, y_test = train_test_split(laptopFeatures, laptopTarget, test_size=0.2, random_state=0)

In [41]:

regressor_input = keras.Input(shape=(X_train.shape[1],))
x = keras.layers.Dense(
    units=10,
    activation='relu',
    use_bias=True,
    bias_initializer="zeros",
)(regressor_input)
regressor_output = keras.layers.Dense(
    units=1,
    activation='relu',
    use_bias=True,
    bias_initializer="zeros",
)(x)
regressor = keras.Model(inputs=regressor_input, outputs=regressor_output)

In [42]:
regressor.summary()
    

In [43]:
regressor.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss=keras.metrics.MSE,
    metrics=[
        keras.metrics.MAE,
        keras.metrics.MSE,
    ],
)

In [44]:
regressor.evaluate(
    x=X_test.astype('float32').values,
    y=y_test.astype('float32').values,
    batch_size=None,
    verbose=1,
)

In [45]:
history = regressor.fit(
    x=X_train.astype('float32').values,
    y=y_train.astype('float32').values,
    batch_size=None,
    epochs=20,
    shuffle=True,
    steps_per_epoch=None,
)

In [46]:


plt.plot(history.history['loss'])
plt.plot(history.history['mean_absolute_error'])